<div class="alert alert-block alert-success">
    
# FIT5196 Task 2: Text Pre-Processing
#### Group_Number: 124
#### Student Name: Pankaj Shitole
#### Student ID: 33570523
#### Student Name: Sachin Shivaramaiah
#### Student ID: 34194037


Date: 30/08/2024


Environment: Google Colab

---

</div>

## Install the required libraries
Install the libraries mentioned below to run the script

In [8]:
# !pip install emoji
# !pip install nltk

## Import the required libraries

In [9]:
import pandas as pd
import json    # To parse the json file from task 1
import os
import re    # Regular expression
import emoji    # To remove the remaining emojis
import nltk    # Text processing
from nltk.stem import PorterStemmer
from collections import defaultdict, Counter
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.util import ngrams
from __future__ import division
from itertools import chain
from nltk.probability import FreqDist
from nltk.probability import *
from nltk.tokenize import RegexpTokenizer

## Step 1: Text extraction
*   You are required to extract the review text for the gmap ids which has at least 70 reviews from the output of task 1



In [10]:
# Load the Task 1 CSV file
csv_df = pd.read_csv('task1_124.csv', encoding='utf-8')

# Filter the gmap_id where review_text_count is greater than or equal to 70
filtered_df = csv_df[csv_df['review_text_count'] >= 70]

# Store the extracted gmap ids
filtered_gmap_ids = filtered_df['gmap_id'].tolist()

In [11]:
len(filtered_gmap_ids)

90

Load the json file since it contains all the reviews. Now we have gmapids with reviews more than 70, we can use that list to fetch the data from the json file.

In [12]:
# Load the Task 1 JSON file
with open('task1_124.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Create a list to store the extracted reviews
extracted_reviews = []

# Iterate over the gmap_ids to extract their review_text
for gmap_id in filtered_gmap_ids:
    if gmap_id in json_data:
        for review in json_data[gmap_id]['reviews']:
            extracted_reviews.append({
                'gmap_id': gmap_id,
                'review_text': review['review_text']
            })

Create a dataframe from the extracted reviews

In [13]:
# Convert the list to a DataFrame
reviews_df = pd.DataFrame(extracted_reviews)

# Remove rows where review_text is None
reviews_df = reviews_df.dropna(subset=['review_text'])
reviews_df = reviews_df[reviews_df['review_text'].str.strip().str.lower() != 'none']  # Remove rows where review_text is "None"

In [14]:
reviews_df

,gmap_id,review_text
0,0x54ce45d613e15fe5:0x44119609f38d89a1,first thing to remember about this place is th...
1,0x54ce45d613e15fe5:0x44119609f38d89a1,"the hotel is old, but the room i got looks ver..."
2,0x54ce45d613e15fe5:0x44119609f38d89a1,my concern is i requested a handicapped access...
3,0x54ce45d613e15fe5:0x44119609f38d89a1,this was a very pleasant stay. the room was co...
4,0x54ce45d613e15fe5:0x44119609f38d89a1,i am going to be honest...i was very impressed...
...,...,...
32292,0x80ea69de56c92db5:0x9c80bcccbaea80cb,coffee and popcorn. awesome.
32293,0x80ea69de56c92db5:0x9c80bcccbaea80cb,love there service
32294,0x80ea69de56c92db5:0x9c80bcccbaea80cb,excellent service!
32295,0x80ea69de56c92db5:0x9c80bcccbaea80cb,great place for oil change


In Task 1, we had some empty reviews because we have removed the emojis from the reviews. Here in task 2, we have to deal with text data and these empty strings doesn't make any sense.

In [15]:
reviews_df[reviews_df['review_text'] == '']

,gmap_id,review_text
1423,0x808572d4362013df:0x41160b5efb0268e9,
3942,0x808580d0d3ac4d49:0xe97b2003c5e06b89,
5389,0x808dff378530903f:0x138b5f6ac3f346d1,
6509,0x808f86a53bbd22eb:0xaff5bdfd04fda402,
8201,0x8094f002c33b5775:0xa2de762d1fba1193,
8728,0x809add6cec217ce7:0xd2c4a926a3f8f3ff,
9798,0x809add6cec217ce7:0xd2c4a926a3f8f3ff,
9942,0x809add6cec217ce7:0xd2c4a926a3f8f3ff,
10045,0x809add6cec217ce7:0xd2c4a926a3f8f3ff,
13074,0x80c2b56685ac2169:0xae5fe05abe26b2,


In [16]:
reviews_df = reviews_df[reviews_df['review_text'] != ""]     # Remove the reviews with empty strings

### Convert all the reviews to lower case

In [17]:
reviews_df.loc[:,'review_text'] = reviews_df['review_text'].apply(lambda x: x.lower() if isinstance(x, str) else x)

### Remove all the emojis

In [18]:
# Function to remove emojis using the emoji library
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

In [19]:
reviews_df.loc[:,'review_text_cleaned'] = reviews_df['review_text'].apply(remove_emojis)    # Remove all the emojis from the text. Reference [1]

<ipython-input-19-ace7938b1ada>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df.loc[:,'review_text_cleaned'] = reviews_df['review_text'].apply(remove_emojis)    # Remove all the emojis from the text. Reference [1]


In [20]:
reviews_df['review_text_cleaned'].value_counts()

,count
review_text_cleaned,
good food,40
good,38
great service,35
great food,25
great,24
...,...
not 8.99,1
worse chinese,1
thanks,1


In [21]:
# Again removing the reviews which are empty strings after removing the emojis
reviews_df = reviews_df[reviews_df['review_text_cleaned'] != ""]

In [22]:
reviews_df

,gmap_id,review_text,review_text_cleaned
0,0x54ce45d613e15fe5:0x44119609f38d89a1,first thing to remember about this place is th...,first thing to remember about this place is th...
1,0x54ce45d613e15fe5:0x44119609f38d89a1,"the hotel is old, but the room i got looks ver...","the hotel is old, but the room i got looks ver..."
2,0x54ce45d613e15fe5:0x44119609f38d89a1,my concern is i requested a handicapped access...,my concern is i requested a handicapped access...
3,0x54ce45d613e15fe5:0x44119609f38d89a1,this was a very pleasant stay. the room was co...,this was a very pleasant stay. the room was co...
4,0x54ce45d613e15fe5:0x44119609f38d89a1,i am going to be honest...i was very impressed...,i am going to be honest...i was very impressed...
...,...,...,...
32292,0x80ea69de56c92db5:0x9c80bcccbaea80cb,coffee and popcorn. awesome.,coffee and popcorn. awesome.
32293,0x80ea69de56c92db5:0x9c80bcccbaea80cb,love there service,love there service
32294,0x80ea69de56c92db5:0x9c80bcccbaea80cb,excellent service!,excellent service!
32295,0x80ea69de56c92db5:0x9c80bcccbaea80cb,great place for oil change,great place for oil change


## Step 2: Generate the unigram and bigram lists and output as vocab.txt

### Step 2.1. Removing non alphabetic characters

In [23]:
tokenizer = RegexpTokenizer(r"[a-zA-Z]+")

In [24]:
def tokenize_reviews_by_gmap_id(reviews_df):
    gmap_unigrams = defaultdict(list)        # To store unigrams per gmap_id
    gmap_bigrams_dict = defaultdict(list)  # To store bigrams per gmap_id

    for index, row in reviews_df.iterrows():
        gmap_id = row['gmap_id']
        review_text = row['review_text']

        # Tokenize the review text into unigrams
        unigram_tokens = tokenizer.tokenize(review_text)

        # Store the unigrams for this review under the respective gmap_id
        gmap_unigrams[gmap_id].extend(unigram_tokens)

        # Extract bigrams within the same review, so it’s easy to back track them to gmap_id
        bigram_tokens = list(ngrams(unigram_tokens, 2))  # Extract bigrams within the review

        # Store the bigrams for this review under the respective gmap_id
        gmap_bigrams_dict[gmap_id].extend(bigram_tokens)

    return gmap_unigrams, gmap_bigrams_dict


In [25]:
gmap_unigrams_dict, gmap_bigrams_dict = tokenize_reviews_by_gmap_id(reviews_df)


### Step 2.2. Generating First 200 meaningful bigrams

Let's create top 200 bigrams

In [26]:
# Use bigram finder to collect the bigrams from unigrams
bigram_measures = BigramAssocMeasures()
bigram_finder = BigramCollocationFinder.from_words(list(gmap_unigrams_dict.values())[0])

In [27]:
# Fetch top 200 bigrams using pmi
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200)
top_200_bigrams


[('another', 'guest'),
 ('answered', 'poor'),
 ('aus', 'sesseln'),
 ('auswahl', 'oder'),
 ('availed', 'themselves'),
 ('awful', 'foam'),
 ('banging', 'slamming'),
 ('bestand', 'neben'),
 ('bett', 'aus'),
 ('bezahlung', 'erfolgt'),
 ('book', 'online'),
 ('both', 'sides'),
 ('bright', 'turquoise'),
 ('broken', 'lamp'),
 ('burn', 'holes'),
 ('bustling', 'metropolitan'),
 ('butter', 'cream'),
 ('cables', 'wiring'),
 ('casual', 'daytrip'),
 ('ceiling', 'seemingly'),
 ('channel', 'selections'),
 ('cheese', 'jam'),
 ('cinnamon', 'swirls'),
 ('closest', 'dealership'),
 ('comfor', 'pup'),
 ('competing', 'places'),
 ('complaints', 'aside'),
 ('cream', 'cheese'),
 ('danishes', 'bagels'),
 ('daughters', 'feet'),
 ('dead', 'ants'),
 ('dem', 'grossen'),
 ('directly', 'under'),
 ('dragging', 'three'),
 ('driver', 'destroying'),
 ('drug', 'den'),
 ('drunk', 'driver'),
 ('dry', 'most'),
 ('dvd', 'player'),
 ('ear', 'plugs'),
 ('eine', 'steckdose'),
 ('eingepacktem', 'geb'),
 ('either', 'direction'),
 (

In [28]:
# Convert top bigrams to a set for quick lookup
top_200_bigrams_set = set(top_200_bigrams)

No need to remove the tokens with length less than length 3 before combining them for bigrams because they might posses some information.
For example, let's consider two unigrams 'not' and 'necessary'. So if we remove the 'not' considering it as a stopword the complete meaning of the context will change. Instead of 'not necessary' it will be considered as necessary


We have to add the respective top 200 bigrams to their respective gmap_ids. So now using the the gmap_bigrams_dict to validate the legit bigrams


In [29]:
# Add these meaningful bigrams (collocations) to the final vocabulary
final_vocab = defaultdict(list)
for gmap_id, unigrams in gmap_unigrams_dict.items():
    # Keep all unigrams in the final vocab
    final_vocab[gmap_id].extend(unigrams)

    # Only add bigrams that are in the top 200 meaningful bigrams
    for bigram in gmap_bigrams_dict[gmap_id]:
        if bigram in top_200_bigrams_set:
            # Convert the bigram tuple to a string and add to the final vocab
            final_vocab[gmap_id].append('_'.join(bigram))


In [30]:
# Display the head of the final vocabulary (first few entries)
final_vocab_head = dict(list(final_vocab.items())[:1])
print("Final Vocabulary with Unigrams and Meaningful Bigrams (Collocations):")
print(final_vocab_head)

Final Vocabulary with Unigrams and Meaningful Bigrams (Collocations):
{'0x54ce45d613e15fe5:0x44119609f38d89a1': ['first', 'thing', 'to', 'remember', 'about', 'this', 'place', 'is', 'that', 'it', 'is', 'old', 'that', 'being', 'said', 'it', 'is', 'in', 'pretty', 'good', 'condition', 'and', 'the', 'owners', 'seem', 'to', 'be', 'taking', 'pride', 'in', 'the', 'appearance', 'the', 'room', 'we', 'were', 'in', 'was', 'very', 'clean', 'the', 'shower', 'was', 'small', 'but', 'there', 'was', 'plenty', 'of', 'hot', 'water', 'microwave', 'fridge', 'and', 'coffee', 'maker', 'in', 'the', 'room', 'was', 'convenient', 'was', 'good', 'value', 'for', 'the', 'price', 'the', 'hotel', 'is', 'old', 'but', 'the', 'room', 'i', 'got', 'looks', 'very', 'clean', 'which', 'is', 'the', 'main', 'thing', 'for', 'me', 'silence', 'warmth', 'and', 'cleanliness', 'thanks', 'to', 'the', 'staff', 'who', 'clean', 'the', 'rooms', 'and', 'the', 'owner', 'good', 'price', 'and', 'quality', 'my', 'concern', 'is', 'i', 'requeste

Check for the lexical diversity and other statistical information of the reviews

In [31]:
words = list(chain.from_iterable(final_vocab.values()))
vocab = set(words)

In [32]:
# Calculate Lexical Diversity
lexical_diversity = len(words)/len(vocab)


In [33]:
print("---Vocab---")
print("Vocabulary size: ",len(vocab),"\nTotal number of tokens: ", len(words), \
"\nLexical diversity: ", lexical_diversity)

---Vocab---
Vocabulary size:  16709 
Total number of tokens:  566531 
Lexical diversity:  33.905739421868454


Let's check for term frequency for the tokens

In [34]:
fd_1 = FreqDist(words)
for i, word in enumerate(fd_1.most_common(25)):
    print(word, end = '\n' if (i+1) % 5 == 0 else ' ')

('the', 23025) ('and', 22580) ('i', 15275) ('to', 14528) ('a', 12148)
('was', 9338) ('is', 7026) ('of', 6943) ('for', 6864) ('my', 6526)
('in', 6227) ('it', 6102) ('they', 5364) ('great', 5341) ('very', 5202)
('with', 5070) ('you', 4831) ('this', 4198) ('that', 3895) ('good', 3882)
('service', 3817) ('food', 3643) ('have', 3528) ('on', 3446) ('me', 3326)


Here after careful observation and after considering the combined token set, we are finding the frequency of each token. From the resultant output in the above cell it is very clear that majority of the frequency are occupied by the stop words. The next step would be removing the stop words.

### Step 2.3. Removal of context-independent stop words

Mount the drive and read the stopwords file

In [35]:
# Mount the drive for stopwords text file
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/Shareddrives/FIT5196_S2_2024/GroupAssessment1/'

Mounted at /content/drive


In [36]:
# Load the provided stopwords from stopwords_en.txt
with open(f'{folder_path}stopwords_en.txt', 'r') as file:
    context_independent_stopwords = set(file.read().splitlines())

In [37]:
len(context_independent_stopwords)

570

Remove the context independent stopwords from the respective gmap_ids

In [38]:
for gmap_id in final_vocab:
    final_vocab[gmap_id] = [token for token in final_vocab[gmap_id] if token.lower() not in context_independent_stopwords]

In [39]:
words_1 = list(chain.from_iterable(final_vocab.values()))
vocab_1 = set(words_1)
# Calculate Lexical Diversity
lexical_diversity_1 = len(words_1)/len(vocab_1)
print("---Vocab after removing context-independent stop words---")
print("Vocabulary size: ",len(vocab_1),"\nTotal number of tokens: ", len(words_1), \
"\nLexical diversity: ", lexical_diversity_1)

---Vocab after removing context-independent stop words---
Vocabulary size:  16224 
Total number of tokens:  237576 
Lexical diversity:  14.643491124260356


### Step 2.4. Removal of context-dependent stop words

In [40]:
# Count the number of gmap_ids in which each word appears
word_gmap_count = Counter()

In [41]:
for tokens in final_vocab.values():
    unique_words = set(tokens)
    word_gmap_count.update(unique_words)

In [42]:
word_gmap_count

Counter({'quick': 71,
         'oj': 3,
         'dealership': 3,
         'alternate_arrangements': 1,
         'thrones': 2,
         'refrigerator': 6,
         'id': 10,
         'scorching': 1,
         'wild_fire': 1,
         'management': 36,
         'hairs': 1,
         'gep': 1,
         'road': 17,
         'mitbekommt': 1,
         'normal_foot': 1,
         'stuffy': 2,
         'realizing': 3,
         'chocolate': 12,
         'full': 63,
         'office': 34,
         'trouble': 20,
         'hauptstrasse_von': 1,
         'teal': 1,
         'press': 3,
         'amenities': 6,
         'black': 20,
         'prickly_pile': 1,
         'cleaning': 24,
         'groggy': 1,
         'care': 62,
         'grinds': 2,
         'sightseeing': 1,
         'personality': 12,
         'personable': 23,
         'parkpl': 1,
         'service': 86,
         'gen_gend': 1,
         'myself_locked': 1,
         'pride': 11,
         'totally': 41,
         'disability': 4,
   

In [43]:
total_gmap = len(final_vocab)
threshold = 0.95 * total_gmap    # calculate the threshold
# Store the words which occur more than in 95% of total businesses
context_dependent_stopwords = {word for word, count in word_gmap_count.items() if count > threshold}

In [44]:
# remove the context-dependent stopwords from the final_vocab
for gmap_id in final_vocab:
    final_vocab[gmap_id] = [token for token in final_vocab[gmap_id] if token.lower() not in context_dependent_stopwords]

Let's print and see how vocab has changed after removing the context dependent stopwords

In [45]:
words_2 = list(chain.from_iterable(final_vocab.values()))
vocab_2 = set(words_2)
# Calculate Lexical Diversity
lexical_diversity_2 = len(words_2)/len(vocab_2)
print("---Vocab after removing context-dependent stop words---")
print("Vocabulary size: ",len(vocab_2),"\nTotal number of tokens: ", len(words_2), \
"\nLexical diversity: ", lexical_diversity_2)

---Vocab after removing context-dependent stop words---
Vocabulary size:  16212 
Total number of tokens:  210223 
Lexical diversity:  12.967123118677522


<div class="alert alert-block alert-warning">
    
### Step 2.5. Remove the tokens with a length less than 3 from the vocab.  <a class="anchor" name="libs"></a>
 </div>

No need to worry about the bigrams because a bigram consists of two words. And each word should be of length 1. So x_y will be of length more than 2.


In [46]:
# Remove tokens with a length less than 3 from the final_vocab
for gmap_id in final_vocab:
    final_vocab[gmap_id] = [token for token in final_vocab[gmap_id] if len(token) >= 3]

# Display the head of the final vocabulary (first few entries) after removing short tokens
final_vocab_head = dict(list(final_vocab.items())[:3])
print("Final Vocabulary after removing tokens with length less than 3:")
print(final_vocab_head)


Final Vocabulary after removing tokens with length less than 3:
{'0x54ce45d613e15fe5:0x44119609f38d89a1': ['thing', 'remember', 'pretty', 'condition', 'owners', 'taking', 'pride', 'appearance', 'room', 'clean', 'shower', 'small', 'plenty', 'hot', 'water', 'microwave', 'fridge', 'coffee', 'maker', 'room', 'convenient', 'price', 'hotel', 'room', 'clean', 'main', 'thing', 'silence', 'warmth', 'cleanliness', 'clean', 'rooms', 'owner', 'price', 'quality', 'concern', 'requested', 'handicapped', 'accessible', 'room', 'hotels', 'hotel', 'room', 'ended', 'regular', 'room', 'clean', 'quiet', 'comfortable', 'smelled', 'bleach', 'cleaning', 'products', 'checked', 'packing', 'car', 'room', 'verify', 'woman', 'looked', 'homeless', 'turned', 'maid', 'concerned', 'maid', 'cart', 'leaving', 'hoped', 'didn', 'mind', 'putting', 'lunch', 'freezer', 'parka', 'matching', 'snow', 'pants', 'pleasant', 'stay', 'room', 'comfortable', 'large', 'plenty', 'warm', 'temp', 'heater', 'efficient', 'walls', 'apparently

In [47]:
words_3 = list(chain.from_iterable(final_vocab.values()))
vocab_3 = set(words_3)
# Calculate Lexical Diversity
lexical_diversity_3 = len(words_3)/len(vocab_3)
print("---Vocab after removing tokens with a length less than 3---")
print("Vocabulary size: ",len(vocab_3),"\nTotal number of tokens: ", len(words_3), \
"\nLexical diversity: ", lexical_diversity_3)


---Vocab after removing tokens with a length less than 3---
Vocabulary size:  16005 
Total number of tokens:  207468 
Lexical diversity:  12.96269915651359


<div class="alert alert-block alert-warning">
    
### Step 2.6. Remove rare tokens.  <a class="anchor" name="libs"></a>
 </div>

Here some bigrams also comes into rare tokens. But we need to retain those bigrams

In [48]:
# Count how many businesses each token appears in
token_business_count = defaultdict(set)

# Iterate over each `gmap_id` and track unique tokens in each business
for gmap_id in final_vocab:
    unique_tokens = set(final_vocab.get(gmap_id, []))  # Get unique tokens from the vocab
    for token in unique_tokens:
        token_business_count[token].add(gmap_id)  # Track in which businesses each token appears


In [49]:
# Calculate the threshold (5% of the total number of businesses in `final_vocab`)
threshold = 0.05 * len(final_vocab)    # Tokens which are in less than 5% of the business

# Identify rare tokens (tokens appearing in fewer businesses than the threshold)
rare_tokens = {token for token, businesses in token_business_count.items() if len(businesses) < threshold}
len(rare_tokens)


12729

In [50]:
rare_bigrams = {token for token in rare_tokens if '_' in token}
len(rare_bigrams)

195

So we need to retain these bigrams for getting removed


In [51]:
# Step to remove rare unigrams from final_vocab
for gmap_id in final_vocab:
    final_vocab[gmap_id] = [
        token for token in final_vocab[gmap_id]
        if (token not in rare_tokens) or ('_' in token)  # Keep bigrams
    ]

In [52]:
words_4 = list(chain.from_iterable(final_vocab.values()))
vocab_4 = set(words_4)
# Calculate Lexical Diversity
lexical_diversity_4 = len(words_4)/len(vocab_4)
print("---Vocab after removing rare tokens---")
print("Vocabulary size: ",len(vocab_4),"\nTotal number of tokens: ", len(words_4), \
"\nLexical diversity: ", lexical_diversity_4)


---Vocab after removing rare tokens---
Vocabulary size:  3471 
Total number of tokens:  171357 
Lexical diversity:  49.36819360414866


<div class="alert alert-block alert-warning">
    
### Step 2.7. Perform stemming on remaining tokens  <a class="anchor" name="libs"></a>
 </div>

Same like previous steps, there no sense in stemming the bigram tokens. So we are just going to stem the unigrams. Similar to the last steps we are using ‘_’ to identify the bigrams and keeping them away from the stemming purpose


In [53]:
stemmer = PorterStemmer()

skipped_bigrams = []

# Stem only unigrams in the final_vocab, and keep track of bigrams that are skipped
for gmap_id in final_vocab:
    updated_tokens = []
    for token in final_vocab[gmap_id]:
        if '_' in token:  # Check if the token is a bigram
            skipped_bigrams.append((gmap_id, token))  # Track the skipped bigram
            updated_tokens.append(token)  # Keep the bigram as it is
        else:
            updated_tokens.append(stemmer.stem(token))  # Stem unigrams

    final_vocab[gmap_id] = updated_tokens  # Update the vocab with the processed tokens


In [54]:
# Count unique bigrams which are skipped
unique_skipped_bigrams = set(bigram for gmap_id, bigram in skipped_bigrams)
num_unique_skipped_bigrams = len(unique_skipped_bigrams)

# Print the count of unique skipped bigrams
print(f"Number of unique bigrams that were skipped: {num_unique_skipped_bigrams}")


Number of unique bigrams that were skipped: 198


<div class="alert alert-block alert-warning">
    
### Step 2.8. Calculate the vocabulary containing both unigrams and bigrams  <a class="anchor" name="libs"></a>
 </div>

In [55]:
words_5 = list(chain.from_iterable(final_vocab.values()))
vocab_5 = set(words_5)
# Calculate Lexical Diversity
lexical_diversity_5 = len(words_5)/len(vocab_5)
print("---Vocab after Stemming---")
print("Vocabulary size: ",len(vocab_5),"\nTotal number of tokens: ", len(words_5), \
"\nLexical diversity: ", lexical_diversity_5)

---Vocab after Stemming---
Vocabulary size:  2447 
Total number of tokens:  171357 
Lexical diversity:  70.02738046587659


Here the vocabulary size is decreased a lot. We have 198 bigrams and remaining unigrams in the vocab.

<div class="alert alert-block alert-warning">
    
### Step 2.9. Create a vocab text file  <a class="anchor" name="libs"></a>
 </div>

In [56]:
# Extract all unique tokens from final_vocab
unique_tokens = set()

for tokens in final_vocab.values():
    unique_tokens.update(tokens)

In [57]:
# Sort tokens alphabetically
sorted_tokens = sorted(unique_tokens)

In [58]:
# Write tokens to a file with their corresponding index
with open("124_vocab.txt", "w") as f:
    for index, token in enumerate(sorted_tokens):
        f.write(f"{token}:{index}\n")

# The vocab.txt file is now created with the required format
print("124_vocab.txt file has been created successfully!")

124_vocab.txt file has been created successfully!


<div class="alert alert-block alert-warning">
    
## Step 3.  Generate the sparse numerical representation and output as countvec.txt <a class="anchor" name="libs"></a>
 </div>

In [59]:
# Create a mapping of tokens to their indexes using the sorted_tokens list
token_to_index = {token: index for index, token in enumerate(sorted_tokens)}

In [60]:
# Count token frequencies for each gmap_id (i.e., for each document) using FreqDist and create the count_vec text file
with open("124_countvec.txt", "w") as f:
    for gmap_id, tokens in final_vocab.items():
        # Use FreqDist to count occurrences of tokens in this gmap_id
        token_counts = FreqDist(tokens)    # Document_frequency

        # Join the tokens' indices with their respective counts
        token_occurrence_str = ",".join(f"{token_to_index[token]}:{count}" for token, count in token_counts.items())

        # Write the gmap_id and token occurrences to the file
        f.write(f"{gmap_id}:{token_occurrence_str}\n")

print("Successfully created count_vec.txt file using FreqDist() function.")

Successfully created count_vec.txt file using FreqDist() function.


## References:


*   [1] https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python



[Link to my workspace](https://colab.research.google.com/drive/1nkjgsiHDdlTXCRkCYdYC6xUIVU3Vlahm?usp=sharing)